In [1]:
import os
from os.path import join as J
import pandas as pd
import numpy as np
import subprocess
from memory_fix import reduce_mem_usage
from tqdm import tqdm

In [2]:
from functions import *

In [3]:
transactions_train = reduce_mem_usage( pd.read_csv('data/transactions_train.csv') )
transactions_test = reduce_mem_usage( pd.read_csv('data/transactions_test.csv') )

Memory usage of dataframe is 668.09 MB
Memory usage after optimization is: 250.53 MB
Decreased by 62.5%
Memory usage of dataframe is 668.38 MB
Memory usage after optimization is: 250.64 MB
Decreased by 62.5%


In [4]:
transactions_train = REN(transactions_train).rename(columns={"amount": "amount_rur"})
transactions_test = REN(transactions_test).rename(columns={"amount": "amount_rur"})

In [5]:
def categories_by_day_cnt(transactions_train):
    X = transactions_train
    categories = [27, 32, 41, 45, 67, 73, 81, 88]

    to_ret = []

    dates = list( set(transactions_train.trans_date) )

    for date in dates:
        to_ret.append( list() )
        for category in categories:
            l = len(X[X.trans_date == date][X.small_group == category] )
            to_ret[-1].append(l)
    DF = pd.DataFrame(to_ret, columns = [str(cat) for cat in categories])
    DF["date"] = dates
    DF = DF.set_index("date")
    return DF

def categories_by_day_sum(transactions_train):
    X = transactions_train
    categories = [27, 32, 41, 45, 67, 73, 81, 88]

    to_ret = []

    dates = list( set(transactions_train.trans_date) )

    for date in dates:
        to_ret.append( list() )
        for category in categories:
            l = np.sum(X[X.trans_date == date][X.small_group == category].amount_rur )
            to_ret[-1].append(l)
    DF = pd.DataFrame(to_ret, columns = [str(cat) for cat in categories])
    DF["date"] = dates
    DF = DF.set_index("date")
    return DF

def categories_by_day_mean(transactions_train):
    X = transactions_train
    categories = [27, 32, 41, 45, 67, 73, 81, 88]

    to_ret = []

    dates = list( set(transactions_train.trans_date) )

    for date in dates:
        to_ret.append( list() )
        for category in categories:
            l = np.mean(X[X.trans_date == date][X.small_group == category].amount_rur )
            to_ret[-1].append(l)
    DF = pd.DataFrame(to_ret, columns = [str(cat) for cat in categories])
    DF["date"] = dates
    DF = DF.set_index("date")
    return DF

In [6]:
la = categories_by_day_cnt(transactions_train[:100])

/home/furfa/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if sys.path[0] == '':


* client_id - уникальный идентификатор клиента
* trans_date - дата совершения транзакции
* small_group - категория покупки
* amount_rur - сумма транзакции

In [7]:
la = la.reset_index()
la[la.date < 0]["45"]

Series([], Name: 45, dtype: int64)

In [8]:
def calc_lag_features(transactions_train, la, lag = 7, category = "27"):
    start_ = get_first_last_transaction_date(transactions_train).last_transaction_date
    end_ = start_ + lag
        
    la = la.reset_index()
    
    
    ids = []
    vals = []
    for i, client_id in enumerate(start_.index):
        
        ids.append(int(client_id))
        
        print(i, end="\r")
        
        vals.append( np.sum(la[la.date > start_[client_id]][ la.date <= end_[client_id] ][category] ) ) 
        # сделать минус транзакции самого пользоателя если играться с валидацией
    
    return pd.DataFrame(np.column_stack([ids, vals]), columns = ["client_id", "val"])

In [9]:
def make_lag_date(transactions_train, categories, lags = [7]):
    
    to_stack = []
    c_id = None
    for lag in lags:
        for category in categories:

            features1 = calc_lag_features(transactions_train, categories_by_day_mean(transactions_train), lag, category)
            features1.columns = ["client_id", "day_mean"]
            print(lag, category, "mean")

            features2 = calc_lag_features(transactions_train, categories_by_day_cnt(transactions_train), lag, category)
            features2.columns = ["client_id", "day_cnt"]
            print(lag, category, "cnt")

            features3 = calc_lag_features(transactions_train, categories_by_day_sum(transactions_train), lag, category)
            features3.columns = ["client_id", "day_sum"]
            print(lag, category, "sum")
            c_id = features2.client_id
            to_stack.append(
                pd.concat([features1["day_mean"], features2["day_cnt"], features3["day_sum"]], axis=1).add_suffix("_"+category).add_suffix("_lag_"+str(lag) )
            )
    return pd.concat([c_id] + to_stack, axis = 1)


In [ ]:
# lag_train = make_lag_date(transactions_train, ['27', '32', '41', '45', '67', '73', '81', '88'], lags = [3, 7])
# lag_train.to_csv("lag_train.csv")
lag_test = make_lag_date(transactions_test, ['27', '32', '41', '45', '67', '73', '81', '88'], lags = [3, 7])
lag_test.to_csv("lag_test.csv")

/home/furfa/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:48: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


/home/furfa/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  app.launch_new_instance()


3 27 mean


/home/furfa/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if sys.path[0] == '':


3 27 cnt


/home/furfa/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


3 27 sum
3 32 mean
3 32 cnt


In [ ]:
lag_test.head()

In [ ]:
def trry(x):
    try:
        return x()
    except:
        return -10000

In [8]:
def get_row_with_max_col_val(df, col, group_col):
    idx = df.groupby(group_col)[col].transform(max) == df[col]
    return df[idx]

# Посмотрим на максимальные - минимальные затраты за 1 день

In [219]:
transactions_train.groupby(['client_id','trans_date'])["amount_rur"].agg(['count', 'sum']).agg(['max', 'min'])

,count,sum
max,96,459306.90625
min,1,0.00100


In [220]:
y_train = REN(pd.read_csv('data/train_target.csv'))

In [221]:
train_preproc = preproc_data(transactions_train, True, y_train)

(25352, 2)
(25000, 210)
(25000, 602)
(25352, 603)
(25352, 612)
(25352, 617)
(25352, 619)


In [222]:
train_preproc.shape, train_preproc.shape

((25000, 1403), (25000, 1403))

In [223]:
transactions_test = reduce_mem_usage(pd.read_csv('data/transactions_test.csv') )
test_id = pd.read_csv('data/test.csv')

Memory usage of dataframe is 668.38 MB
Memory usage after optimization is: 250.64 MB
Decreased by 62.5%


In [224]:
test_id.shape

(25000, 1)

In [225]:
transactions_test  = REN(transactions_test).rename(columns={"amount": "amount_rur"})

In [226]:
transactions_test

,client_id,trans_date,amount_rur,small_group
0,31453,0,20.533001,0
1,3615,0,7.700000,0
2,19950,0,13.441000,1
3,7311,0,8.675000,7
4,36112,0,23.459000,2
...,...,...,...,...
21901475,37497,119,3.850000,7
21901476,35648,119,24.150999,15
21901477,632,119,67.782997,11
21901478,4848,119,13.860000,45


In [227]:
test_id = REN(test_id)

In [228]:
test = preproc_data(transactions_test, False, test_id = test_id)
test.shape

(25388, 2)
(25000, 197)
(25000, 393)
(25000, 785)
(25388, 786)
(25388, 795)
(25388, 800)
(25388, 802)


(25000, 1586)

In [229]:
common_features = list(set(train_preproc.columns).intersection(set(test.columns)))

In [230]:
len(common_features)

1194

In [248]:
train_preproc = train_preproc.merge(lag_train, on="client_id")

In [256]:
TO_PREDICT = ['27', '32', '41', '45', '67', '73', '81', '88']

train_target = train_preproc[['client_id'] + TO_PREDICT]
train_data = train_preproc[common_features]
X_test = test[common_features]

In [258]:
train_data = pd.concat([train_data, train_preproc[lag_train.columns] ], axis=1)
del train_data["client_id"]

In [238]:
from sklearn.model_selection import train_test_split, cross_validate, cross_val_score
import catboost as cb

# 5000 k iters
seed 228 - 0.6419666666666667 0,6393
seed 123213 - 0.6421666666666667

In [241]:
TO_PREDICT = ['27', '32', '41', '45', '67', '73', '81', '88']
def average_score(model, columns, train_data, y_train):
    scores = []
    for col in columns:
        score = cross_val_score(
            model,
            train_data,
            y_train[col],
            cv = 4,
            scoring = "roc_auc",
            verbose = 1,
        )
        print(col, score, np.mean(score))
        scores.append( np.mean(score) )
    return np.mean(scores)

In [242]:
score = average_score(
    cb.CatBoostClassifier(iterations = 500, logging_level="Silent"),
    ["88"],
    train_data,
    y_train
)
print(score)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


88 [0.74111194 0.82545764 0.76241806 0.75917083] 0.772039615888877
0.772039615888877


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:  2.3min finished


In [260]:
lag_train.to_csv("lag_train_cirk.csv")

In [262]:
model = cb.CatBoostClassifier(iterations = 500)
model.fit(train_data, y_train["88"])

Learning rate set to 0.077261
0:	learn: 0.5559426	total: 74.9ms	remaining: 37.4s
1:	learn: 0.4501335	total: 149ms	remaining: 37s
2:	learn: 0.3695931	total: 223ms	remaining: 36.9s
3:	learn: 0.3073152	total: 304ms	remaining: 37.8s
4:	learn: 0.2600258	total: 379ms	remaining: 37.5s
5:	learn: 0.2228670	total: 454ms	remaining: 37.4s
6:	learn: 0.1951524	total: 528ms	remaining: 37.2s
7:	learn: 0.1707640	total: 601ms	remaining: 37s
8:	learn: 0.1542508	total: 678ms	remaining: 37s
9:	learn: 0.1413217	total: 751ms	remaining: 36.8s
10:	learn: 0.1301967	total: 826ms	remaining: 36.7s
11:	learn: 0.1199129	total: 901ms	remaining: 36.6s
12:	learn: 0.1121087	total: 975ms	remaining: 36.5s
13:	learn: 0.1068217	total: 1.05s	remaining: 36.4s
14:	learn: 0.1025636	total: 1.12s	remaining: 36.3s
15:	learn: 0.0983523	total: 1.2s	remaining: 36.2s
16:	learn: 0.0953937	total: 1.27s	remaining: 36.1s
17:	learn: 0.0921896	total: 1.35s	remaining: 36.1s
18:	learn: 0.0899325	total: 1.42s	remaining: 36s
19:	learn: 0.087968

In [265]:
import eli5


In [294]:
imp = pd.DataFrame([model.feature_importances_,model.feature_names_]).T

In [270]:
eli5.explain_weights_catboost(model, top=300)

Weight,Feature
0.0583,small_group_std_88
0.0463,small_group_88
0.0241,small_group_median_88
0.0190,"sg_sum_('sum', 88)"
0.0179,small_group_31
0.0168,small_group_sum_88
0.0162,"sg_sum_('max', 31)"
0.0158,small_group_std_31
0.0152,monthcount_diff
0.0126,small_group_std_6


In [259]:
score = average_score(
    cb.CatBoostClassifier(iterations = 500, logging_level="Silent"),
    ["88"],
    train_data,
    y_train
)
print(score)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


88 [0.74718826 0.80835406 0.76769214 0.76167818] 0.7712281634244003
0.7712281634244003


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:  2.3min finished


In [243]:
lag_train.head(1)

,client_id,day_mean_88_lag_7,day_cnt_88_lag_7,day_sum_88_lag_7
0,0,0.0,0,0.0


In [ ]:
score = average_score(
    cb.CatBoostClassifier(iterations = 500, logging_level="Silent"),
    ["88"],
    train_data,
    y_train
)
print(score)

In [59]:
def predict(model):
    pred = []
    for ind, col in enumerate(TO_PREDICT):
        print(f"{ind+1}/{len(TO_PREDICT)}")
        
        model.fit(train_data, y_train[col])
        proba = model.predict_proba(X_test)[:,1]
        pred.append(proba)
    
    return dict(zip(TO_PREDICT, pred))

In [60]:
prediction = predict(cb.CatBoostClassifier())

Learning rate set to 0.040724
0:	learn: 0.6596293	total: 169ms	remaining: 2m 48s
1:	learn: 0.6299906	total: 317ms	remaining: 2m 38s
2:	learn: 0.6056154	total: 447ms	remaining: 2m 28s
3:	learn: 0.5816168	total: 571ms	remaining: 2m 22s
4:	learn: 0.5579121	total: 708ms	remaining: 2m 20s
5:	learn: 0.5391811	total: 833ms	remaining: 2m 17s
6:	learn: 0.5206290	total: 961ms	remaining: 2m 16s
7:	learn: 0.5047884	total: 1.12s	remaining: 2m 18s
8:	learn: 0.4917457	total: 1.25s	remaining: 2m 18s
9:	learn: 0.4774788	total: 1.37s	remaining: 2m 16s
10:	learn: 0.4641897	total: 1.51s	remaining: 2m 15s
11:	learn: 0.4532841	total: 1.64s	remaining: 2m 15s
12:	learn: 0.4452653	total: 1.79s	remaining: 2m 16s
13:	learn: 0.4377068	total: 1.92s	remaining: 2m 15s
14:	learn: 0.4303802	total: 2.06s	remaining: 2m 14s
15:	learn: 0.4240304	total: 2.22s	remaining: 2m 16s
16:	learn: 0.4166387	total: 2.36s	remaining: 2m 16s
17:	learn: 0.4106308	total: 2.49s	remaining: 2m 15s
18:	learn: 0.4058260	total: 2.62s	remaining:

In [75]:
prediction

{'27': array([[0.95744376, 0.04255624],
        [0.96868047, 0.03131953],
        [0.96667636, 0.03332364],
        ...,
        [0.97697982, 0.02302018],
        [0.97227382, 0.02772618],
        [0.44328191, 0.55671809]]), '32': array([[0.82711677, 0.17288323],
        [0.95105991, 0.04894009],
        [0.95724851, 0.04275149],
        ...,
        [0.57358712, 0.42641288],
        [0.95236974, 0.04763026],
        [0.76039027, 0.23960973]]), '41': array([[0.78331357, 0.21668643],
        [0.91822824, 0.08177176],
        [0.93122326, 0.06877674],
        ...,
        [0.91273796, 0.08726204],
        [0.9512628 , 0.0487372 ],
        [0.86372021, 0.13627979]]), '45': array([[0.75339982, 0.24660018],
        [0.54239124, 0.45760876],
        [0.40129667, 0.59870333],
        ...,
        [0.7340702 , 0.2659298 ],
        [0.45796309, 0.54203691],
        [0.77855048, 0.22144952]]), '67': array([[0.9883229 , 0.0116771 ],
        [0.99330594, 0.00669406],
        [0.99361267, 0.0063873

In [48]:
average_score()

NameError: name 'average_score' is not defined

In [90]:
def submit(prediction, filename):
    submission = pd.DataFrame(data=np.zeros((25000,8)),columns=train_target.columns[1:],index=test_id['client_id'].values)
    submission.columns = ['cat_27','cat_32','cat_41','cat_45','cat_67','cat_73','cat_81','cat_88']
    
    for ind in TO_PREDICT:
        submission['cat_'+ind] = prediction[ind][:,1]
    
    submission.index.name = 'client_dk'
    submission.to_csv("f{filename}.csv", index=True)

In [53]:
cross_val_score(
    cb.CatBoostClassifier(logging_level="Silent"),
    train_data,
    y_train["27"],
    cv = 4,
    scoring = "accuracy",
    verbose = 1,
)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:  8.4min finished


array([0.85410334, 0.85330347, 0.85837734, 0.85517683])

In [108]:
param={'objective':'multi:softprob','num_class':4,'n_jobs':16,'seed':42}
model_xgb = xgb.XGBClassifier(**param,n_estimators=600)